In [1]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

pd.options.display.float_format = '{:20,.5f}'.format  ### Setting pd to have a numerical precision up to 5 decimal points

In [2]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_sample = df_scaled.sample(n=10000, replace=False, random_state=42)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

## Default SE on the original OLS is HC3, default CI on bootstrapped parameter is BC. 

That is by default se_type = 'hc3', ci_type = 'bc'.

The default seed value is None. For reproducability, use an integer of your choice. 

In [4]:
psb = PairsBootstrap(Y_data, X_data, reps = 1000, se_type='hc3', ci_type = 'bc', seed = 42)  # seed = None, se_type = 'hc3' and ci_type = 'bc' are the default options for these arguments.
psb.fit()

In [5]:
rsb = ResidualBootstrap(Y_data, X_data, reps = 1000, seed = 42)
rsb.fit()

In [6]:
wb_unif = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "uniform", seed = 42)
wb_unif.fit()

In [7]:
wb_stdn = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "standard_normal", seed = 42)
wb_stdn.fit()

In [8]:
wb_rad = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "rademacher", seed = 42)
wb_rad.fit()

In [9]:
wb_webb4 = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "webb4", seed = 42)
wb_webb4.fit()

In [10]:
wb_webb6 = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "webb6", seed = 42)
wb_webb6.fit()

In [11]:
wb_cont_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen_cont", seed = 42)
wb_cont_mam.fit()

In [12]:
wb_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen", seed = 42)
wb_mam.fit()

In [13]:
# Now: np.ndarray was supported as an input array. pd.Series, list, and tuples were also added to the supported list for an input array. 
# Before that only pd.DataFrame had been supported. 
X_data_np = X_data.to_numpy()
Y_data_np = Y_data.to_numpy()

In [14]:
wb_mam_np = WildBootstrap(Y_data_np, X_data_np, reps = 1000, from_distro = "mammen", seed = 42)
wb_mam_np.fit()

In [15]:
psb.summary()

+---------------------------------------------------------------------------------------------------------------------+
|               Pairs Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI               |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2070    | 0.0000 |     0.0454    |    0.0462    |    -1.70     | [6.1273, 6.3045] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1691    | 0.0002 |     0.0050    |    0.0051    |    -1.56     | [0.1591, 0.1785] |
+--------------+------------+-----------

In [16]:
rsb.summary()

+---------------------------------------------------------------------------------------------------------------------+
|              Residual Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI             |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2058    | 0.0012 |     0.0454    |    0.0349    |    23.00     | [6.1420, 6.2788] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1694    | 0.0001 |     0.0050    |    0.0039    |    22.09     | [0.1617, 0.1772] |
+--------------+------------+-----------

In [17]:
wb_unif.summary()

+---------------------------------------------------------------------------------------------------------------------+
|         Wild Bootstrap with Uniform results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI         |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2075    | 0.0005 |     0.0454    |    0.0345    |    24.04     | [6.1371, 6.2737] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1693    | 0.0000 |     0.0050    |    0.0039    |    21.25     | [0.1622, 0.1776] |
+--------------+------------+-----------

In [18]:
wb_stdn.summary()

+---------------------------------------------------------------------------------------------------------------------+
|     Wild Bootstrap with Standard Normal results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI     |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2063    | 0.0007 |     0.0454    |    0.0358    |    21.13     | [6.1328, 6.2748] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1693    | 0.0000 |     0.0050    |    0.0040    |    20.59     | [0.1617, 0.1771] |
+--------------+------------+-----------

In [19]:
wb_rad.summary()

+---------------------------------------------------------------------------------------------------------------------+
|        Wild Bootstrap with Rademacher results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI       |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2061    | 0.0008 |     0.0454    |    0.0345    |    23.98     | [6.1416, 6.2784] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1693    | 0.0001 |     0.0050    |    0.0039    |    22.74     | [0.1616, 0.1766] |
+--------------+------------+-----------

In [20]:
wb_webb4.summary()

+---------------------------------------------------------------------------------------------------------------------+
|          Wild Bootstrap with Webb4 results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI          |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2077    | 0.0007 |     0.0454    |    0.0346    |    23.65     | [6.1365, 6.2756] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1692    | 0.0001 |     0.0050    |    0.0039    |    21.56     | [0.1618, 0.1776] |
+--------------+------------+-----------

In [21]:
wb_webb6.summary()

+---------------------------------------------------------------------------------------------------------------------+
|          Wild Bootstrap with Webb6 results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI          |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2076    | 0.0007 |     0.0454    |    0.0347    |    23.60     | [6.1386, 6.2751] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1693    | 0.0000 |     0.0050    |    0.0039    |    21.92     | [0.1618, 0.1773] |
+--------------+------------+-----------

In [22]:
wb_cont_mam.summary()

+---------------------------------------------------------------------------------------------------------------------+
|       Wild Bootstrap with Mammen Cont results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI       |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2065    | 0.0005 |     0.0454    |    0.0357    |    21.43     | [6.1414, 6.2805] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1694    | 0.0001 |     0.0050    |    0.0040    |    20.39     | [0.1611, 0.1768] |
+--------------+------------+-----------

In [23]:
wb_mam.summary()

+---------------------------------------------------------------------------------------------------------------------+
|          Wild Bootstrap with Mammen results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI         |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2074    | 0.0005 |     0.0454    |    0.0356    |    21.61     | [6.1341, 6.2722] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1692    | 0.0001 |     0.0050    |    0.0040    |    19.52     | [0.1616, 0.1773] |
+--------------+------------+-----------

Same output for wb_mam_np as in the case for wb_mam, but the difference is the independent variables' name. wb_mam was constructed with pd.DataFrame the input which holds information about the column name. 
When it comes to wb_mam_np, it was constructed with np.ndarray as the input which DOES NOT hold info about the column name originally. Because of this, x1, x2, ...., xn were generated as alternative independent variable names. 

In [24]:
wb_mam_np.summary()

+--------------------------------------------------------------------------------------------------------------+
|      Wild Bootstrap with Mammen results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI      |
+-------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|  Var  | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+-------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| const |   6.2070   |     6.2074    | 0.0005 |     0.0454    |    0.0356    |    21.61     | [6.1341, 6.2722] |
+-------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|   x1  |   0.1693   |     0.1692    | 0.0001 |     0.0050    |    0.0040    |    19.52     | [0.1616, 0.1773] |
+-------+------------+---------------+--------+---------------+--------------+--------------+---

## Some useful methods were implemented (let's use it on wb_mam object of Wild Bootstrap with Mammen)

- The common in the following three methods is that either a string (if one wishes to capture one variable or ci) or 1-D like array can be provided to 'which_var' and/or 'which_ci' (the latter if exists in that object class).

### get_ci() method

#### Vanila version is when only the actual CI was used with all independent variables

In [25]:
wb_mam.get_ci()

bc                     
                              lwb                  upb
const                     6.13409              6.27220
tanass_clean              0.16162              0.17733
tax                       0.51603              0.54219

#### However, any combination of ('bc', 'bca', 'percentile) CI types could be selected and any combinations of independent variables can be chosen with 'which_ci' and 'which_var' optional arguments, respectively.

Please note that if choosing 'bca' the calculation can take a while as it uses jacknife resampling for calculating the acceleration factor

In [26]:
wb_mam.get_ci(which_ci='all', which_var='all')

bc                                       bca  \
                              lwb                  upb                  lwb   
const                     6.13409              6.27220              6.13412   
tanass_clean              0.16162              0.17733              0.16170   
tax                       0.51603              0.54219              0.51587   

                                            percentile                       
                              upb                  lwb                  upb  
const                     6.27221              6.13621              6.27331  
tanass_clean              0.17734              0.16156              0.17723  
tax                       0.54214              0.51649              0.54232

In [27]:
wb_mam.get_ci(which_ci=['bc', 'percentile'], which_var=['tax', 'const'])

bc                                percentile  \
                       lwb                  upb                  lwb   
tax                0.51603              0.54219              0.51649   
const              6.13409              6.27220              6.13621   

                            
                       upb  
tax                0.54232  
const              6.27331

In [28]:
wb_mam.get_ci(which_ci=['bca', 'bc'], which_var='tanass_clean')

bc                                       bca  \
                              lwb                  upb                  lwb   
tanass_clean              0.16162              0.17733              0.16170   

                                   
                              upb  
tanass_clean              0.17734

In [29]:
wb_mam.get_ci(which_ci='bc', which_var='tanass_clean')

bc                     
                              lwb                  upb
tanass_clean              0.16162              0.17733

### get_all_se() method

The following SE-s are calculated: 
- bootstrapped - standard error of the bootstrapped parameters
- constant - model-based OLS Standard Errors, that is, constant variance is assumed 
- HC0, HC1, HC2, HC3, HC4, HC4m, HC5 - Heteroskedasticity-Consistent Standard Errors (HCE) using sandwich estimators 

#### Vanila version is when using all indepencdent variables. 

In [30]:
wb_mam.get_all_se()

,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5,bootstrapped
const,0.03532,0.04536,0.04536,0.04537,0.04538,0.04538,0.04538,0.04538,0.03557
tanass_clean,0.00391,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499,0.00402
tax,0.00657,0.00910,0.00910,0.00911,0.00911,0.00911,0.00911,0.00911,0.00657


#### A subset of indepent variables can be chosen with 'which_var' argument to calculate the above-mentioned 9 SE-s

In [31]:
wb_mam.get_all_se(which_var=['tanass_clean', 'tax'])

,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5,bootstrapped
tanass_clean,0.00391,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499,0.00402
tax,0.00657,0.00910,0.00910,0.00911,0.00911,0.00911,0.00911,0.00911,0.00657


In [32]:
wb_mam.get_all_se(which_var='tax')

,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5,bootstrapped
tax,0.00657,0.00910,0.00910,0.00911,0.00911,0.00911,0.00911,0.00911,0.00657


### get_bootstrap_params() method

#### Vanila version: Returning a dataframe capturing the parameter estimate of ALL each independent variables in each (wild) bootstrap.

In [33]:
wb_mam.get_bootstrap_params()

,const,tanass_clean,tax
0,6.11794,0.17610,0.53383
1,6.21305,0.16659,0.53469
2,6.23408,0.16652,0.53049
3,6.21177,0.16390,0.53824
4,6.24215,0.16528,0.52969
...,...,...,...
995,6.23701,0.17162,0.51927
996,6.20005,0.17196,0.53056
997,6.21973,0.16943,0.53008
998,6.17875,0.16621,0.53192


#### As usual, the desired independent variable can be chosen with 'which_var' argument

In [34]:
wb_mam.get_bootstrap_params(which_var='tax')

,tax
0,0.53383
1,0.53469
2,0.53049
3,0.53824
4,0.52969
...,...
995,0.51927
996,0.53056
997,0.53008
998,0.53192


In [35]:
wb_mam.get_bootstrap_params(which_var=('const', 'tanass_clean'))

,const,tanass_clean
0,6.11794,0.17610
1,6.21305,0.16659
2,6.23408,0.16652
3,6.21177,0.16390
4,6.24215,0.16528
...,...,...
995,6.23701,0.17162
996,6.20005,0.17196
997,6.21973,0.16943
998,6.17875,0.16621


### bp_test() and white_test() methods

#### Vanila version: for bp_test() returning the robust version (proposed by Roger Koenker) of this test when the presence of the heteroscedasticity is examined on the residual of the OLS on the original sample data. For white_test(), there are no optional arguments

In [36]:
wb_mam.bp_test()

(432.43914611970746,
 1.2503527125935506e-94,
 225.92457000184137,
 1.083837549448663e-96)

In [37]:
wb_mam.white_test()

(1437.6088470921256, 9.758682033770634e-309, 335.59463848972536, 0.0)

#### The non-robust version of bp_test() can be achieved by setting bp_test(robust = False).

In [38]:
wb_mam.bp_test(robust = False)

(1035.8816259850282,
 1.1512322968569012e-225,
 225.92457000184234,
 1.083837549446753e-96)

At each test, the first value is the calculated LM-test stats, the second is the 'LM p-value' corresponding to the LM-test stats, the third value is the F-test stats and last one is the 'F p-value'.

As it can be seen the p-values are 0, so we can rejcet the null hypothesis of homoscedasticity, and accept the alternative hypothesis that the variance of the residuals is heteroscedastic.

### Some built-in attributes

In [39]:
wb_mam.indep_varname

['const', 'tanass_clean', 'tax']

#### wb_mam.bs_params and wb_mam.get_bootstrap_params() basically output the same thing with the exception that the latter one is transposed and is in pd.DataFrame.

In [40]:
wb_mam.bs_params

array([[6.11793735, 6.21304945, 6.23407723, ..., 6.21972686, 6.17874968,
        6.26229539],
       [0.17609648, 0.16658936, 0.16651714, ..., 0.16943475, 0.16621383,
        0.17268631],
       [0.53383021, 0.53468577, 0.53049401, ..., 0.53007589, 0.53192339,
        0.51534074]])

In [41]:
wb_mam.orig_params

array([6.20695725, 0.16929278, 0.52929771])

In [42]:
wb_mam.bs_mean

array([6.20741928, 0.16919201, 0.52934914])

In [43]:
wb_mam.orig_se

array([0.0453789 , 0.00499137, 0.00910921])

In [44]:
wb_mam.bs_se

array([0.03557226, 0.00401694, 0.00656538])

In [45]:
wb_mam.bs_ci

array([[6.13409065, 6.27219828],
       [0.16162345, 0.17733217],
       [0.51603411, 0.54218543]])